In [8]:
# import os

# os.makedirs('going_modular')

In [9]:
# 1. get the data
import requests
import zipfile
from pathlib import Path


URL = "https://github.com/mrdbourke/pytorch-deep-learning/raw/refs/heads/main/data/pizza_steak_sushi.zip"
# Setup path to a data folder
data_path = Path('data/')
image_path = data_path / 'pizza_steak_sushi'


# If the imgae folder doesn't exist, download it and prepare it
if image_path.is_dir():
    print(f'{image_path} directory already exists... skipping download')
else:
    print(f'{image_path} directory does not exist... creating one....')
    image_path.mkdir(parents=True, exist_ok=True)


# Download pizza, steak, and sushi data
with open(data_path/ 'pizza_steak_sushi.zip', 'wb') as f:
    request = requests.get(URL)
    print("Downloading pizza, steak and sushi data...")
    f.write(request.content)
    print("Download done.......")


# Unzip pizza, steak and sushi data
with zipfile.ZipFile(data_path / 'pizza_steak_sushi.zip', 'r') as zip_ref:
    print("Unzipping pizza, steak and sushi data")
    zip_ref.extractall(image_path)
    print("Extracted All....")

data/pizza_steak_sushi directory already exists... skipping download
Download done.......
Unzipping pizza, steak and sushi data
Extracted All....


In [10]:
# 2. Become one with the data
import os
def walk_through_dir(dir_path):
  """Walks through dir_path returning file counts of its contents."""
  for dirpath, dirnames, filenames in os.walk(dir_path):
    print(f"There are {len(dirnames)} directories and {len(filenames)} images in '{dirpath}'.")

walk_through_dir(image_path)

There are 2 directories and 0 images in 'data/pizza_steak_sushi'.
There are 3 directories and 0 images in 'data/pizza_steak_sushi/test'.
There are 0 directories and 25 images in 'data/pizza_steak_sushi/test/pizza'.
There are 0 directories and 19 images in 'data/pizza_steak_sushi/test/steak'.
There are 0 directories and 31 images in 'data/pizza_steak_sushi/test/sushi'.
There are 3 directories and 0 images in 'data/pizza_steak_sushi/train'.
There are 0 directories and 78 images in 'data/pizza_steak_sushi/train/pizza'.
There are 0 directories and 75 images in 'data/pizza_steak_sushi/train/steak'.
There are 0 directories and 72 images in 'data/pizza_steak_sushi/train/sushi'.


In [11]:
# Setup train and testing paths
train_dir = image_path / 'train'
test_dir = image_path / 'test'

train_dir, test_dir

(PosixPath('data/pizza_steak_sushi/train'),
 PosixPath('data/pizza_steak_sushi/test'))

In [12]:
# Transform the data
import torch
from torch.utils.data import DataLoader
from torchvision import datasets, transforms

image_data_transform = transforms.Compose([transforms.Resize(size=(64,64)), 
                                           transforms.RandomHorizontalFlip(p=0.5),
                                           transforms.ToTensor()])


In [13]:
%%writefile going_modular/data_setup.py
"""
Contains functionality for creating Pytorch DataLoader's for Image classification data
"""
import os
from torchvision import datasets, transforms
from torch.utils.data import DataLoader

NUM_WORKERS = os.cpu_count()

def create_dataloaders(
    train_dir: str,
    test_dir: str,
    transform: transforms.Compose,
    batch_size: int,
    num_workers: int=NUM_WORKERS
):
    """Creates training and testing DataLoaders.
    
    Takes in a training directory and testing directory path and turns them into
    PyTorch Datasets & then into PyTorch DataLoaders.
    
    Args:
        train_dir: Path to train data directory.
        test_dir: Path to test data directory.
        transform: torchvision transforms to perform on training and testing data.
        batch_size: Number of sample per batch in each of the DataLoaders.
        num_workers: An integar for number of workers per DataLoader.
    
    Returns:
        A tuple of (train_dataloader, test_dataloader, class_names).
        Where class_names is a list of the target classes.
    
    Examples:
        train_dataloader, test_dataloader, class_names = create_dataloaders(train_dir="path/to/train_dir",
            test_dir="path/to/test_dir",
            transform=some_transform,
            batch_size=32,
            num_workers=4)
    """

    # Use ImageFolder to create dataset(s)
    train_data = datasets.ImageFolder(root=train_dir, transform=transform, target_transform=None)
    test_data = datasets.ImageFolder(root=test_dir, transform=transform, target_transform=None)

    # Get the class names
    class_names = train_data.classes

    # Turn datasets into DataLoaders
    train_dataloader = DataLoader(dataset=train_data, batch_size=batch_size, shuffle=True, num_workers=num_workers, pin_memory=True)
    test_dataloader = DataLoader(dataset=test_data, batch_size=batch_size, shuffle=False, num_workers=num_workers, pin_memory=True)
    
    return train_dataloader, test_dataloader, class_names


Overwriting going_modular/data_setup.py


In [14]:
from going_modular import data_setup

train_dataloader, test_dataloader, class_names = data_setup.create_dataloaders(train_dir=train_dir, 
                                                                               test_dir=test_dir, 
                                                                               transform=image_data_transform, 
                                                                               batch_size=32, num_workers=4)
train_dataloader, test_dataloader, class_names

(<torch.utils.data.dataloader.DataLoader at 0x7f4e1b551010>,
 ['pizza', 'steak', 'sushi'])

In [15]:
%%writefile going_modular/model_builder.py
"""
Contains PyTorch model code to instantiate a TinyVGG Model. 
"""

import torch
from torch import nn

class TinyVGG(nn.Module):
    """Creates the TinyVGG architecture

    Replicates the TinyVGG architechture from the CNN explainer website in PyTorch
    The original architecture here: https://poloclub.github.io/cnn-explainer/

    Args:
        input_shape: An integar indicating number of input channels.
        hidden_units: An integar indicating number of hidden units between layers.
        output_shape: An integar indicating number of output units.
    """
    def __init__(self, input_shape: int, hidden_units: int, output_shape: int):
        super().__init__()
        self.conv_block_1 = nn.Sequential(
            nn.Conv2d(in_channels=input_shape, 
                      out_channels=hidden_units, 
                      kernel_size=3, 
                      stride=1, 
                      padding=0),
            nn.ReLU(),
            nn.Conv2d(in_channels=hidden_units, 
                      out_channels=hidden_units, 
                      kernel_size=3, 
                      stride=1, padding=0),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2)
        )
        self.conv_block_2 = nn.Sequential(
            nn.Conv2d(in_channels=hidden_units, 
                      out_channels=hidden_units, 
                      kernel_size=3, 
                      stride=1, 
                      padding=0),
            nn.ReLU(),
            nn.Conv2d(in_channels=hidden_units, 
                      out_channels=hidden_units, 
                      kernel_size=3, 
                      stride=1, 
                      padding=0),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2,
                         stride=2)
        )
        self.classifier = nn.Sequential(
            nn.Flatten(),
            nn.Linear(in_features=hidden_units*13*13, 
                      out_features=output_shape)
        )

    def forward(self, x: torch.Tensor):
        return self.classifier(self.conv_block_2(self.conv_block_1(x)))

Overwriting going_modular/model_builder.py


In [16]:
import torch
from going_modular import model_builder


device = 'cuda' if torch.cuda.is_available() else 'cpu'

# create an instance of the tinyvgg model from the script
torch.manual_seed(42)
model_0 = model_builder.TinyVGG(input_shape=3, hidden_units=10, output_shape=len(class_names)).to(device)

model_0


TinyVGG(
  (conv_block_1): Sequential(
    (0): Conv2d(3, 10, kernel_size=(3, 3), stride=(1, 1))
    (1): ReLU()
    (2): Conv2d(10, 10, kernel_size=(3, 3), stride=(1, 1))
    (3): ReLU()
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (conv_block_2): Sequential(
    (0): Conv2d(10, 10, kernel_size=(3, 3), stride=(1, 1))
    (1): ReLU()
    (2): Conv2d(10, 10, kernel_size=(3, 3), stride=(1, 1))
    (3): ReLU()
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (classifier): Sequential(
    (0): Flatten(start_dim=1, end_dim=-1)
    (1): Linear(in_features=1690, out_features=3, bias=True)
  )
)

In [17]:
# 1. Get a batch of images and labels from the DataLoader
img_batch, label_batch = next(iter(train_dataloader))

# 2. Get a single image from the batch and unsqueeze the image so its shape fits the model
img_single, label_single = img_batch[0].unsqueeze(dim=0), label_batch[0]
print(f"Single image shape: {img_single.shape}\n")

# 3. Perform a forward pass on a single image
model_0.eval()
with torch.inference_mode():
    pred = model_0(img_single.to(device))
    
# 4. Print out what's happening and convert model logits -> pred probs -> pred label
print(f"Output logits:\n{pred}\n")
print(f"Output prediction probabilities:\n{torch.softmax(pred, dim=1)}\n")
print(f"Output prediction label:\n{torch.argmax(torch.softmax(pred, dim=1), dim=1)}\n")
print(f"Actual label:\n{label_single}")

Single image shape: torch.Size([1, 3, 64, 64])

Output logits:
tensor([[ 0.0208, -0.0019,  0.0095]], device='cuda:0')

Output prediction probabilities:
tensor([[0.3371, 0.3295, 0.3333]], device='cuda:0')

Output prediction label:
tensor([0], device='cuda:0')

Actual label:
0


In [2]:
%%writefile going_modular/engine.py
"""
Contains functions for training and testing a PyTorch model
"""

import torch
from torch.utils.tensorboard import SummaryWriter
import torchmetrics
from tqdm.auto import tqdm
from typing import Dict, List, Tuple

def train_step(model: torch.nn.Module, 
               dataloader: torch.utils.data.DataLoader, 
               loss_fn: torch.nn.Module, 
               optimizer: torch.optim.Optimizer,
               accuracy: torchmetrics.classification.accuracy.Accuracy,
               device: torch.device) -> Tuple[float, float]:
    """Trains a PyTorch model for a single epoch
    
    Turns a target PyTorch model to training model and then runs
    through all of the required training step (forward pass, 
    loss claculation, optimizer step).

    Args:
        model: A PyTorch model to be trained.
        dataloader: A DataLoader instance for the model to be trained on.
        loss_fn: A PyTorch loss function to be minimized.
        optimizer: A PyTorch optimizer to help minimize the loss function.
        accuracy: A torchmetric module to calculate accuracy.
        device: A target device to compute on (i.e. "cuda" or "cpu")

    Returns:
        A tuple of training loss and training accuracy metrics.
        In the form (train_loss, train_accuracy). For example:

        (0.1112, 0.8743)
    """
    # Put the model in train mode.
    model.train()

    # Setup train loss and train accuracy
    train_loss, train_accuracy = 0, 0

    # Loop through the dataloader data batches
    for batch, (X,y) in enumerate(dataloader):
        # Send data to target device
        X, y = X.to(device), y.to(device)

        # 1. Forward Pass
        y_logits = model(X)

        # 2. Claculate loss and accumulate loss
        loss = loss_fn(y_logits, y)
        # print(f"loss: \n {loss}")
        train_loss += loss.item()

        # 3. Optimizer zero grad
        optimizer.zero_grad()

        # 4. Loss backward
        loss.backward()

        # 5. Optimizer step
        optimizer.step()

        # Calculate and accumulate accuracy metric accross all batches
        y_pred = torch.argmax(torch.softmax(y_logits, dim=1), dim=1)
        accuracy_value = accuracy(y_pred, y)
        # print(f"accuracy: \n {accuracy}")
        train_accuracy += accuracy_value.item()

    # Adjust metrics to get average loss and accuracy per batch
    train_loss = train_loss / len(dataloader)
    train_accuracy = train_accuracy / len(dataloader)

    return train_loss, train_accuracy
    

def test_step(model: torch.nn.Module, 
              dataloader: torch.utils.data.DataLoader, 
              loss_fn: torch.nn.Module, 
              accuracy: torchmetrics.classification.accuracy.Accuracy,
              device: torch.device) -> Tuple[float, float]:
    """Tests a PyTorch model for a single epoch
    
    Turns a target PyTorch model to "eval" mode and then performs
    forward pass on testing dataset and also calculate testing loss
    and testing accuracy.

    Args:
        model: A PyTorch model to be tested.
        dataloader: A DataLoader instance for the model to be tested on.
        loss_fn: A PyTorch loss function to calculate loss on the test data.
        accuracy: A torchmetric module to calculate accuracy.
        device: A target device to compute on (i.e. "cuda" or "cpu")

    Returns:
        A tuple of testing loss and testing accuracy metrics.
        In the form (test_loss, test_accuracy). For example:

        (0.0112, 0.9343)
    """
    # Put the model in eval mode
    model.eval()

    # Setup the test loss and test accuracy
    test_loss, test_accuracy = 0, 0

    # Turn on inference context manager
    with torch.inference_mode():
        # Loop through DataLoader batchs
        for batch, (X,y) in enumerate(dataloader):
            # Send data to a target device
            X, y = X.to(device), y.to(device)

            # 1. Forward pass
            y_logits = model(X)

            # 2. Calculate and accumulate loss
            loss = loss_fn(y_logits, y)
            test_loss += loss.item()

            # Calculate accumulate accuracy
            y_pred = torch.argmax(torch.softmax(y_logits, dim=1), dim=1)
            accuracy_value = accuracy(y_pred, y)
            test_accuracy += accuracy_value.item()
    
    # Adjust metrics to get average loss and accuracy per batch
    test_loss = test_loss / len(dataloader)
    test_accuracy = test_accuracy / len(dataloader)

    return test_loss, test_accuracy
    

def train(model: torch.nn.Module, 
          train_dataloader: torch.utils.data.DataLoader, 
          test_dataloader: torch.utils.data.DataLoader, 
          loss_fn: torch.nn.Module, 
          optimizer: torch.optim.Optimizer,
          accuracy: torchmetrics.classification.accuracy.Accuracy,
          epochs: int,
          device: torch.device, 
          writer: torch.utils.tensorboard.writer.SummaryWriter) -> Dict[str, List]:
    """Trains and test a PyTorch model

    Passes a target PyTorch model through train_step() and test_step()
    functions for a number of epochs. training and testing the model in the same epoch loop.

    Calculates, prints and stores evaluation metrics throughout.
    
    Stores metrics to specified writer log_dir if present.
    
    Args:
        model: A PyTorch model to be tested.
        train_dataloader: A DataLoader instance for the model to be trained on.
        test_dataloader: A DataLoader instance for the model to be tested on.
        loss_fn: A PyTorch loss function to calculate loss on the test data.
        optimizer: A PyTorch optimizer to help minimize the loss function.
        accuracy: A torchmetric module to calculate accuracy.
        epochs: An integar indicating how many epochs to train for.
        device: A target device to compute on (i.e. "cuda" or "cpu").
        writer: A SummaryWriter() instance to log model results to.


    Returns:
        A dictionary of training and testing loss as well as training and
        testing accuracy metrics. Each metric has a value in a list for 
        each epoch.
        In the form: {train_loss: [...],
                      train_acc: [...],
                      test_loss: [...],
                      test_acc: [...]} 
        For example if training for epochs=2: 
                     {train_loss: [2.0616, 1.0537],
                      train_acc: [0.3945, 0.3945],
                      test_loss: [1.2641, 1.5706],
                      test_acc: [0.3400, 0.2973]} 
  """
    # Create empty results dictionary
    results = { "train_loss": [], 
                "train_accuracy": [], 
                "test_loss": [], 
                "test_accuracy": []
              }

    # Loop through training and testing steps for a number of epochs
    for epoch in tqdm(range(epochs)):
        train_loss, train_accuracy = train_step(model=model, 
                                                dataloader=train_dataloader, 
                                                loss_fn=loss_fn, 
                                                optimizer=optimizer, 
                                                accuracy=accuracy, 
                                                device=device)
        test_loss, test_accuracy = test_step(model=model, 
                                             dataloader=test_dataloader, 
                                             loss_fn=loss_fn,
                                             accuracy=accuracy, 
                                             device=device)
        
        print(
            f"Epoch: {epoch+1} | "
            f"train_loss: {train_loss: .4f} | "
            f"train_accuracy: {train_accuracy: .4f} | "
            f"test_loss: {test_loss: .4f} | "
            f"test_accuracy: {test_accuracy: .4f}"
        )
        # 5. update the results
        results["train_loss"].append(train_loss)
        results["train_accuracy"].append(train_accuracy)
        results["test_loss"].append(test_loss)
        results["test_accuracy"].append(test_accuracy)

        #### New: Experiment tracking with tensorboard ####
        if writer:
            writer.add_scalars(main_tag="Loss", 
                               tag_scalar_dict={"train_loss": train_loss, 
                                                "test_loss": test_loss}, 
                               global_step=epoch)
            
            writer.add_scalars(main_tag="Accuracy", 
                               tag_scalar_dict={"train_accuracy": train_accuracy, 
                                                "test_accuracy": test_accuracy}, 
                               global_step=epoch)
    
            writer.add_graph(model=model, input_to_model=torch.randn(32,3,224,224).to(device))
    
            # Close the writer
            writer.close()
        else:
            pass

        #### End: Experiment tracking with tensorboard ####
        

    return results


Overwriting going_modular/engine.py


In [1]:
%%writefile going_modular/utils.py
"""
Contains various utility functions for PyTorch model training and saving.
"""

import os
from datetime import datetime
import torch
from torch.utils.tensorboard import SummaryWriter
import matplotlib.pyplot as plt
from pathlib import Path
from typing import Dict, List


def save_model(model: torch.nn.Module, 
               target_dir: str,
               model_name: str):
    """Saves a PyTorch model to a target directory
    
    Args:
        model: A target PyTorch model to save.
        target_dir: A directory for saving the model to.
        model_name: A filename for the saved model. Should include either ".pth" or ".pt" as the file extension.

    Example Usage:
        save_model(model=model_1, 
                   target_dir='models', 
                   model_name='tiny_vgg_model.pth')    
    """
    # Create target directory
    target_dir_path = Path(target_dir)
    target_dir_path.mkdir(parents=True, exist_ok=True)

    # Create model save path
    assert model_name.endswith(".pth") or model_name.endswith(".pt"), "model_name should end with '.pth' or '.pth'"
    model_save_path = target_dir_path / model_name

    # Save the model state_dict()
    print(f"[INFO] Saving model to: {model_save_path}")
    torch.save(obj=model.state_dict(), f=model_save_path)


def create_writer(experiment_name: str, 
                  model_name: str, 
                  extra: str = None) -> torch.utils.tensorboard.writer.SummaryWriter():
    """Create a torch.utils.tensorboard.writer.SummaryWriter() instance saving to a specific log_dir
    
    log_dir is a combination of runs/timestamp/experiment_name/model_name/extra

    Where timestamp is current date in YYYY-MM-DD format

    Args:
        experiment_name (str): Name of the experiment.
        model_name (str): Name of the model
        extra (str, optional): Anything extra to add to the directory.

    Returns:
        torch.utils.tensorboard.writer.SummaryWriter(): Instance of a writer saving to the specific log_dir.

    Example usage:
        # Create a writer saving to "runs/2022-06-04/data_10_percent/effnetb2/5_epochs/"
        writer = create_writer(experiment_name="data_10_percent",
                               model_name="effnetb2",
                               extra="5_epochs")
        # The above is the same as:
        writer = SummaryWriter(log_dir="runs/2022-06-04/data_10_percent/effnetb2/5_epochs/")
    """

    # Get timestamp of current date in reverse order (YYYY-MM-DD)
    timestamp = datetime.now().strftime("%Y-%b-%d")

    if extra:
        # create log directory path
        log_dir = os.path.join('runs', timestamp, experiment_name, model_name, extra)
    else:
        log_dir = os.path.join('runs', timestamp, experiment_name, model_name)

    print(f"[INFO] Created SummaryWriter saving to {log_dir}")
    return SummaryWriter(log_dir=log_dir)
    




def plot_loss_curves(results: Dict[str, List[float]]):
    """Plots training curves of a results dictionary
    Args:
        results (dict): dictionary containing list of values, e.g.
            {"train_loss": [...],
             "train_accuracy": [...],
             "test_loss": [...],
             "test_accuracy": [...]}
    """
    # Get the loss values from the results dictionary (training and testing)
    loss = results["train_loss"]
    test_loss = results["test_loss"]

    # Get the accuracy values from the results dictionary (training & testing)
    accuracy = results["train_accuracy"]
    test_accuracy = results["test_accuracy"]

    # Get the epochs
    epochs = range(len(results["train_loss"]))

    # Setup the plot
    plt.figure(figsize=(15,7))

    # plot the loss
    plt.subplot(1,2,1)
    plt.plot(epochs, loss, label="Train Loss")
    plt.plot(epochs, test_loss, label="Test Loss")
    plt.title("Loss Curves")
    plt.xlabel("epochs")
    plt.ylabel("loss")
    plt.legend()

    # plot the loss
    plt.subplot(1,2,2)
    plt.plot(epochs, accuracy, label="Train Accuracy")
    plt.plot(epochs, test_accuracy, label="Test Accuracy")
    plt.title("Accuracy Curves")
    plt.xlabel("epochs")
    plt.ylabel("accuracy")
    plt.legend()

Overwriting going_modular/utils.py


In [20]:
%%writefile going_modular/train.py
"""
Trains a PyTorch image classification model using device agnostic code.
"""

import os
import torch
from torchvision import transforms
from torchmetrics import Accuracy
from timeit import default_timer as timer

import data_setup, engine, model_builder, utils


NUM_EPOCHS = 5
BATCH_SIZE = 32
HIDDEN_UNITS = 10
LEARNING_RATE = 0.001


# Setup directories
train_dir = "data/pizza_steak_sushi/train"
test_dir = "data/pizza_steak_sushi/test"

# Setup device agnostic code
device = "cuda" if torch.cuda.is_available() else "cpu"

# Create tranform
data_transform = transforms.Compose([
    transforms.Resize(size=(64,64)),
    transforms.ToTensor()
])

# Create DataLoaders with help of data_setup.py
train_dataloader, test_dataloader, class_names = data_setup.create_dataloaders(train_dir=train_dir, 
                                                                               test_dir=test_dir, 
                                                                               transform=data_transform, 
                                                                               batch_size=BATCH_SIZE, 
                                                                               num_workers=os.cpu_count())

# Create model with help of mode_builder.py
model = model_builder.TinyVGG(input_shape=3, hidden_units=HIDDEN_UNITS, output_shape=len(class_names)).to(device)

# Set Loss function, Optimizer, Accuracy
loss_fn = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(params=model.parameters(), lr=LEARNING_RATE)
accuracy = Accuracy(task='multiclass', num_classes=len(class_names)).to(device)


# Start the timer
start_time = timer()

# Start training with help of engine.py
engine.train(model=model, 
             train_dataloader=train_dataloader, 
             test_dataloader=test_dataloader, 
             loss_fn=loss_fn, 
             optimizer=optimizer, 
             accuracy=accuracy, 
             epochs=NUM_EPOCHS, 
             device=device)

# End timer
end_time = timer()
print(f"[INFO] Total training time: {end_time-start_time:.3f} seconds")


# Save the model with help of utils.py
utils.save_model(model=model, target_dir="models", model_name="tiny_vgg_model_v0.pth")


Overwriting going_modular/train.py


In [23]:
!python going_modular/train.py

100%|█████████████████████████████████████████████| 5/5 [00:04<00:00,  1.09it/s]
[INFO] Total training time: 4.585 seconds
[INFO] Saving model to: models/tiny_vgg_model_v0.pth
